In [57]:
import zlib
import base64
import duckdb as ddb
import ast


In [30]:
ddb.connect("../data/memory.duckdb")
ddb.execute("INSTALL FLOCKMTL FROM community")
ddb.execute("LOAD FLOCKMTL")

In [ ]:
ddb.execute("""DROP SECRET groq""")
ddb.execute("""
                CREATE SECRET groq (
                    TYPE OPENAI,
                    BASE_URL 'https://api.groq.com/openai/v1/',
                    API_KEY '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'
                );  
                """)

ddb.sql("""DELETE MODEL 'mermaid_generator';""")
ddb.execute("""
                CREATE MODEL(
                    'mermaid_generator',
                    'qwen-2.5-coder-32b',
                    'openai',
                    {"context_window": 128000, "max_output_tokens": 32000}
                );  
                """)

ddb.execute("""DELETE PROMPT 'mermaid_generator_prompt';""")
ddb.execute("""
                CREATE PROMPT('mermaid_generator_prompt', 'Génère le code mermaid, et rien que le code dans le champ json "mermaid_code", expliquant le mieux la description qui suit, et avec un effort d originalité. Fais en sorte que les noms des entités mermaid soient concis et évite les caractères qui feraient bugger le script mermaid. Dans le champ mermaid_diag_type tu vas indiquer le type de diagramme le plus adapté que tu as choisi ou que l utilisateur t a demandé.')
                """)

ddb.sql("""GET MODELS;""")

ddb.sql("""FROM duckdb_secrets();""")

ddb.sql("""GET PROMPTS;""")

In [113]:
input_text="""

Le syndicat élabore et tient à jour un plan régional de transport, qui définit les services de transports publics de personnes réguliers et à la demande, les services de transport scolaire et les services de transport fluvial régulier de personnes qu'il organise en application des articles L. 1241-1 et L. 1241-2.

Le syndicat inscrit chacun de ces services au plan régional de transport en précisant sa consistance. La conclusion d'un contrat de service public pour l'exploitation de services de transports publics de voyageurs mentionnés à l'article L. 1241-1 vaut inscription au plan régional de transport.

Les décisions de modification ou de suppression des services inscrits au plan suivent le même régime. Pour les services de transport de voyageurs mentionnés à l'article L. 1241-1 faisant l'objet d'un contrat de service public conclu après le 3 décembre 2009, la fin du contrat vaut suppression de l'inscription au plan régional de transport.

Le syndicat peut déléguer l'inscription au plan régional de transport aux autorités mentionnées à l'article R. 1241-38. Celles-ci mettent à jour le plan régional de transport conformément aux dispositions de l'article R. 1241-39.

Le syndicat s'assure de la cohérence et veille à la coordination de l'ensemble des services inscrits au plan régional de transport.

"""

query = """
with llm_result as (
    select 
    llm_complete_json(
        {'model_name': 'mermaid_generator','secret_name': 'groq' }, 
        {'prompt_name': 'mermaid_generator_prompt', 'version': 1}, 
        {'prompt_input': ?}
    )::json AS json_result
) 
select json_result.mermaid_code, json_result.mermaid_diag_type from llm_result
"""

res = ddb.execute(query, [input_text]).fetchdf()

In [114]:
res

,mermaid_code,mermaid_diag_type
0,"""graph TD;\nA[Syndicat] --> B[Plan Régional de...","""Diagramme de flux"""


In [115]:
mermaid_code = ast.literal_eval(res["mermaid_code"][0])
mermaid_type = ast.literal_eval(res["mermaid_diag_type"][0])


In [110]:
def kroki_url_from_mermaid(mermaid_code: str, output_format: str = "svg") -> str:
    compressed = zlib.compress(mermaid_code.encode('utf-8'))
    b64 = base64.urlsafe_b64encode(compressed).decode('utf-8')
    return f"https://kroki.io/mermaid/{output_format}/{b64}"

In [116]:
kroki_url_from_mermaid(mermaid_code)

'https://kroki.io/mermaid/svg/eJx1z0sKwjAQBuC9p8jOlScQBG191AcU6y50EZPYDoQkTFLBI_UcvZhpbAsK7kLy5Z_5K2S2Jrd0OVvT4qUFcOZLslisyIbmimly7doKjGaKCEluyLSzBn25nG2iSmgh8Qlcuq93kjd3BdxNLv3jCm4UA5QT3P6BO9U8gangdjQx2iPzPRjwMG_YPJSJhz1Nu5aDC_vHtIsR8OgbhotyRIcfVDTWonTuYzK6brxB8F3rpvRj_0XJKgYRMc-04wg2xkaTjeGnsGvdtSh12FGETmO5afw5CIMC9BD2Zd7_a41h'